In [1]:
import pandas as pd
import janitor as jn
import numpy as np
from janitor.functions._numba import (
    _numba_multiple_non_equi_join,
    _get_regions_non_equi,
    _align_indices_and_regions,
    _get_indices_dual_non_monotonic_non_equi,
)

In [2]:
%load_ext line_profiler

In [3]:
url = 'https://raw.githubusercontent.com/samukweku/data-wrangling-blog/master/notebooks/Data_files/results.csv'
url = '/Users/samuel.oranyeli/events.csv'
events = pd.read_csv(url, parse_dates=['start', 'end']).iloc[:, 1:]
events.head()

,name,audience,start,sponsor,end
0,Event 1,1178,2022-11-19 10:00:00,Sponsor 2,2022-11-19 10:15:00
1,Event 2,1446,2015-09-27 15:00:00,Sponsor 11,2015-09-27 15:11:00
2,Event 3,2261,2019-11-12 18:00:00,Sponsor 10,2019-11-12 18:53:00
3,Event 4,1471,2019-12-24 22:00:00,Sponsor 6,2019-12-24 22:11:00
4,Event 5,2605,2028-06-20 12:00:00,Sponsor 8,2028-06-20 12:31:00


In [4]:
(events
.conditional_join(
    events,
    ('start', 'end', '<='),
    ('end', 'start', '>='),
    # ('id', 'id', '!='),
    use_numba = True,
    df_columns = ['start', 'end'],
    right_columns = ['start', 'end'])
)

(array([28282, 28281, 28280, ...,     2,     1,     0]),
 array([28282, 28281, 28280, ...,     2,     1,     0]),
 array([13367, 15884,  3544, ..., 24103, 10726, 25296]),
 array([13367, 15884,  3544, ..., 24103, 10726, 25296]),
 array([    0,     1,     2, ..., 29996, 29997, 29998]),
 29999,
 array([28282, 28281, 28280, ...,     2,     1,     0]),
 4)

In [5]:
print 'i'

SyntaxError: Missing parentheses in call to 'print'. Did you mean print(...)? (414025362.py, line 1)

In [ ]:
# %%timeit
# (events
# .conditional_join(
#     events,
#     ('start', 'end', '<='),
#     ('end', 'start', '>='),
#     # ('id', 'id', '!='),
#     use_numba = False,
#     df_columns = ['start', 'end'],
#     right_columns = ['start', 'end'])
# )

In [6]:

np.random.seed(0)
dd = pd.DataFrame({'value':np.random.randint(100000, size=50_000)})
df = pd.DataFrame({'start':np.random.randint(100000, size=1_000),
                         'end':np.random.randint(100000, size=1_000)})

In [7]:

(
            left_region,
            right_region,
            left_index,
            right_index,
            starts,
            ends,
            cum_max_arr,
            max_freq
        ) = dd.conditional_join(
    df, ("value", "start", ">"), ("value", "end", "<"), use_numba=True
)

In [8]:
%lprun -f _get_indices_dual_non_monotonic_non_equi.__wrapped__ _get_indices_dual_non_monotonic_non_equi.__wrapped__(left_region, right_region, left_index, right_index, starts, cum_max_arr, ends,max_freq)

Timer unit: 1e-09 s

Total time: 51.2866 s
File: /Users/samuel.oranyeli/pyjanitor/janitor/functions/_numba.py
Function: _get_indices_dual_non_monotonic_non_equi at line 1034

Line #      Hits         Time  Per Hit   % Time  Line Contents
  1034                                           @njit()
  1035                                           def _get_indices_dual_non_monotonic_non_equi(
  1036                                               left_region: np.ndarray,
  1037                                               right_region: np.ndarray,
  1038                                               left_index: np.ndarray,
  1039                                               right_index: np.ndarray,
  1040                                               starts: np.ndarray,
  1041                                               max_arr: np.ndarray,
  1042                                               ends: int,
  1043                                               max_freq:int
  1044               

In [ ]:
left_region

array([980, 980, 980, ...,   0,   0,   0])

In [ ]:
cum_max_arr

array([980, 980, 980, 980, 980, 980, 980, 980, 980, 980, 980, 980, 980,
       980, 980, 980, 980, 980, 980, 980, 980, 980, 980, 980, 980, 980,
       980, 980, 980, 980, 980, 980, 980, 980, 980, 980, 980, 980, 980,
       980, 980, 980, 980, 980, 980, 980, 980, 980, 980, 980, 980, 980,
       980, 980, 980, 980, 980, 980, 980, 980, 980, 980, 980, 980, 980,
       980, 980, 980, 980, 980, 980, 980, 980, 980, 980, 980, 980, 980,
       980, 980, 980, 980, 980, 980, 980, 980, 980, 980, 980, 980, 980,
       980, 980, 980, 980, 980, 980, 980, 980, 980, 980, 980, 980, 980,
       980, 980, 980, 980, 980, 980, 980, 980, 980, 980, 980, 980, 980,
       980, 980, 980, 980, 980, 980, 980, 980, 980, 980, 980, 980, 980,
       980, 980, 980, 980, 980, 980, 980, 980, 980, 980, 980, 980, 980,
       980, 980, 980, 980, 980, 980, 980, 980, 980, 980, 980, 980, 980,
       980, 980, 980, 980, 980, 980, 980, 980, 980, 980, 980, 980, 980,
       980, 980, 980, 980, 980, 980, 980, 980, 980, 980, 980, 98

In [16]:
right_region.max()

980

In [13]:
%timeit pd.Index(right_region).nunique()

32.8 µs ± 178 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [ ]:
C=dict(key=[f"c{num}" for num in range(1,8)],
       vol=[35,15,5,35,18,90,17],
       profit=[45,35,55,12,15,55,11],
       unitsSold=[15,10,30,10,15,80,2]
       )
C=pd.DataFrame(C)


D=dict(key=[f'd{num}' for num in range(1,9)],
       vol=[20,50,15,16,40,20,40,2],
       profit=[30,10,12,52,35,20,30,57],
       unitsSold=[20,35,10,12,40,30,5,15]
       )
D=pd.DataFrame(D)

In [ ]:
C.conditional_join(D,('vol','vol','<'),('profit','profit','>'),use_numba=True)

(array([0, 3, 1, 2, 3, 2]),
 array([1, 3, 3, 2, 2, 0, 3]),
 array([2, 0, 1, 4, 3, 6]),
 array([3, 4, 6, 0, 5, 2, 1]),
 array([0, 1, 2, 5, 6, 6]),
 7,
 array([3, 3, 3, 3, 3, 3, 3]),
 3)

In [ ]:
right_region

array([356,  58, 424, 695,  73, 386, 721, 152, 765, 435,  90, 946, 741,
       159, 451, 215, 363, 496, 627, 792, 677, 910, 949, 132, 480, 273,
       813, 341, 632, 171, 711, 899, 916, 632, 698, 585, 489, 854, 596,
        87, 205, 456, 230, 604, 718, 834, 683, 142, 145, 798, 667, 953,
       216, 616, 607, 925, 161, 642, 586, 826, 370, 719, 975,  35, 441,
       635, 473, 781, 430, 894, 892, 800,   1, 401, 878, 539, 265, 248,
       117, 937,  77, 592, 569, 848, 936, 821, 421, 106, 828, 412,  32,
       688, 458, 125, 127, 131, 823,  16, 300, 371, 851, 969,  50, 753,
       348, 708, 340, 749, 212, 771, 776, 209, 129, 399, 942, 203, 923,
       402, 359, 220, 403, 814, 713, 471, 382, 746, 486,  13,  46, 582,
       767, 319, 605, 517, 554, 450, 672, 932, 922, 442, 208, 434, 358,
       288, 231, 214, 121, 950, 559, 499, 871, 390,  74, 295, 534, 482,
       696, 913, 681, 511, 201, 429, 368, 613, 847, 885, 448, 709, 886,
       362, 291, 773, 190, 890, 948, 452, 911, 253, 645, 921, 57

In [ ]:
# (array([22093, 31192, 29739, ...,   501,  5468, 12591]),
#  array([  8,   8,   8, ..., 158, 158, 158]))